In [1]:
import random     
import numpy as np
import tensorflow as tf

from TicTacToe import TicTacToe
from MCTS import MCTS
from simulate_self_play_games import simulate_self_play_games
from head_to_head_match import head_to_head_match
from TrainingManager import TrainingManager

from Config import *

#tf.compat.v1.disable_eager_execution()

2023-08-01 13:11:44.030536: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-01 13:11:44.081089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 13:11:44.808282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
manager = TrainingManager()
manager.train()

2023-08-01 13:11:47.058258: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-08-01 13:11:47.058293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: drew-lenovo
2023-08-01 13:11:47.058299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: drew-lenovo
2023-08-01 13:11:47.058401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.5
2023-08-01 13:11:47.058419: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.5
2023-08-01 13:11:47.058424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.5




--------- Episode  0  ---------
Simulating games...
68 examples generated.

Training on examples:
 Minibatch Policy Acc: 0.45588234066963196, Value Acc: 0.6617646813392639
New network against old network (testing for improvement): 
W/D/L: 10  /  0  /  10
Maintaining old network.  Can't compete with the best
Best network against MCTS: 
W/D/L: 5  /  2  /  13
Total episode time:  92.65458154678345


--------- Episode  1  ---------
Simulating games...
67 examples generated.

Training on examples:
 Minibatch Policy Acc: 0.5820895433425903, Value Acc: 0.5223880410194397
New network against old network (testing for improvement): 
W/D/L: 20  /  0  /  0
Replacing network!  Great success


INFO:tensorflow:Assets written to: Networks/Episode_1/assets


INFO:tensorflow:Assets written to: Networks/Episode_1/assets


Best network against MCTS: 
W/D/L: 11  /  0  /  9
Total episode time:  90.78642582893372


In [5]:
"""
search = MCTS()

while True:
    game_over, value = search.board.is_game_over()
    if game_over:
        search.board.print()
        break
    
    search.search()
    print(search.n)
    print(search.pi)
    print(search.board.get_legal_moves())
    child_choice = np.argmax(search.pi)
    #child_choice = np.random.choice(len(search.pi), p=search.pi)
    print(child_choice)
    move_choice = search.children[child_choice].move
    print(move_choice)
    search = search.make_move(move_choice)
    search.board.print()
"""


100
[0.12, 0.12, 0.12, 0.09, 0.12, 0.11, 0.12, 0.1, 0.1]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
0
0
 _  _  _ 

 _  _  _     O's move

 _  _  x 



112
[0.11607142857142858, 0.13392857142857142, 0.11607142857142858, 0.14285714285714285, 0.13392857142857142, 0.125, 0.125, 0.10714285714285714]
[1, 2, 3, 4, 5, 6, 7, 8]
3
4
 _  _  _ 

 _  o  _     X's move

 _  _  x 



116
[0.15517241379310345, 0.15517241379310345, 0.15517241379310345, 0.14655172413793102, 0.13793103448275862, 0.11206896551724138, 0.13793103448275862]
[1, 2, 3, 5, 6, 7, 8]
0
1
 _  _  _ 

 _  o  _     O's move

 _  x  x 



118
[0.2033898305084746, 0.1694915254237288, 0.16101694915254236, 0.1864406779661017, 0.13559322033898305, 0.1440677966101695]
[2, 3, 5, 6, 7, 8]
0
2
 _  _  _ 

 _  o  _     X's move

 o  x  x 



124
[0.20161290322580644, 0.1935483870967742, 0.27419354838709675, 0.1532258064516129, 0.1774193548387097]
[3, 5, 6, 7, 8]
2
6
 _  _  x 

 _  o  _     O's move

 o  x  x 



134
[0.39552238805970147, 0.22388059701492538, 